# Assignment 3

## Question 3

##### Write a python subroutine that takes in as inputs the number of links and the DH parameters in table/matrix form, and returns the (a) complete manipulator Jacobian, (b) the end-effector position, and (c) end-effector velocity. If you need any other inputs (such as information about the nature of joints (R/P), incorporate this as an additional input to the python code. However, the code is to be setup in a way that if this information is not provided, default assumption of all joints being revolute joints is to be assumed.

a) Complete Manipulator Jacobian

In [13]:
import numpy as np

def dh_transform_matrix(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha),  np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta),  np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [            0,              np.sin(alpha),              np.cos(alpha),              d],
        [            0,                          0,                          0,              1]
    ])

def calculate_jacobian(num_links, dh_params, joint_types=None):
    if joint_types is None:
        joint_types = ['R'] * num_links 
    
    J = np.zeros((6, num_links))  
    T = np.eye(4)  

    for i in range(num_links):
        theta, d, a, alpha = dh_params[i]
        T_link = dh_transform_matrix(theta, d, a, alpha)
        T = np.dot(T, T_link)
        
        if joint_types[i] == 'R':
            J[:3, i] = np.cross(T[:3, 2], T[:3, 3] - np.eye(4)[:3, 3])
            J[3:, i] = T[:3, 2]
        else:
            J[:3, i] = T[:3, 2]
            J[3:, i] = np.zeros(3)

    return J


b) End - Effector Position

In [14]:
def end_effector_position(num_links, dh_params):
    T = np.eye(4) 

    for i in range(num_links):
        theta, d, a, alpha = dh_params[i]
        T_link = dh_transform_matrix(theta, d, a, alpha)
        T = np.dot(T, T_link)

    return T[:3, 3]


c) End - Effector Veloccity

In [15]:
def end_effector_velocity(num_links, dh_params, joint_velocities, joint_types=None):
    J = calculate_jacobian(num_links, dh_params, joint_types)
    velocity = np.dot(J, joint_velocities)
    return velocity


In [16]:
num_links = 3
dh_params = np.array([
    [np.pi/2, 0, 1, np.pi/2],
    [0, 0, 1, 0],
    [np.pi/2, 0, 1, np.pi/2]
])
joint_velocities = np.array([0.5, 0.5, 0.5])

Jacobian = calculate_jacobian(num_links, dh_params)
end_position = end_effector_position(num_links, dh_params)
end_velocity = end_effector_velocity(num_links, dh_params, joint_velocities)

# Display results
print("Jacobian:\n", Jacobian)
print()
print("End-Effector Position:\n", end_position)
print()
print("End-Effector Velocity:\n", end_velocity)
print()


Jacobian:
 [[-6.12323400e-17 -1.22464680e-16  1.00000000e+00]
 [ 3.74939946e-33  7.49879891e-33 -1.22464680e-16]
 [ 1.00000000e+00  2.00000000e+00  1.83697020e-16]
 [ 1.00000000e+00  1.00000000e+00  1.22464680e-16]
 [-6.12323400e-17 -6.12323400e-17  1.00000000e+00]
 [ 6.12323400e-17  6.12323400e-17 -6.12323400e-17]]

End-Effector Position:
 [6.123234e-17 2.000000e+00 1.000000e+00]

End-Effector Velocity:
 [ 5.000000e-01 -6.123234e-17  1.500000e+00  1.000000e+00  5.000000e-01
  3.061617e-17]



## Question 4

#### Apply the above code to the two common RRP configurations of Stanford manipulator and SCARA manipulator. Verify that the results obtained using the code match with the expressions derived earlier (by yourself and in the textbook). You may choose a few configurations (numerical values) to verify your results.

Stanford Manipulator (RRP Configuration)

In [17]:
num_links_stanford = 3
dh_params_stanford = np.array([
    [0, 0, 2, np.pi/2],   # Link 1
    [0, 0, 0, -np.pi/2],  # Link 2
    [0, 1, 0, 0]          # Link 3 (Prismatic)
])
joint_types_stanford = ['R', 'R', 'P']


In [18]:
Jacobian_stanford = calculate_jacobian(num_links_stanford, dh_params_stanford, joint_types_stanford)
end_position_stanford = end_effector_position(num_links_stanford, dh_params_stanford)
end_velocity_stanford = end_effector_velocity(num_links_stanford, dh_params_stanford, [0.5, 0.5, 0.5], joint_types_stanford)

print("Stanford Manipulator Jacobian:\n", Jacobian_stanford)
print()
print("Stanford Manipulator End-Effector Position:\n", end_position_stanford)
print()
print("Stanford Manipulator End-Effector Velocity:\n", end_velocity_stanford)


Stanford Manipulator Jacobian:
 [[-0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 1.2246468e-16  2.0000000e+00  0.0000000e+00]
 [ 2.0000000e+00  0.0000000e+00  1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 6.1232340e-17  1.0000000e+00  0.0000000e+00]]

Stanford Manipulator End-Effector Position:
 [2. 0. 1.]

Stanford Manipulator End-Effector Velocity:
 [ 0.   1.   1.5  0.  -0.5  0.5]


SCARA Manipulator (RRP Configuration)

In [19]:
num_links_scara = 3
dh_params_scara = np.array([
    [0, 0.5, 1, 0],     
    [0, 0, 1, 0],        
    [0, 0, 0, 0]         
])
joint_types_scara = ['R', 'R', 'P']

In [20]:
Jacobian_scara = calculate_jacobian(num_links_scara, dh_params_scara, joint_types_scara)
end_position_scara = end_effector_position(num_links_scara, dh_params_scara)
end_velocity_scara = end_effector_velocity(num_links_scara, dh_params_scara, [0.5, 0.5, 0.5], joint_types_scara)

print("SCARA Manipulator Jacobian:\n", Jacobian_scara)
print()
print("SCARA Manipulator End-Effector Position:\n", end_position_scara)
print()
print("SCARA Manipulator End-Effector Velocity:\n", end_velocity_scara)


SCARA Manipulator Jacobian:
 [[0. 0. 0.]
 [1. 2. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 0.]]

SCARA Manipulator End-Effector Position:
 [2.  0.  0.5]

SCARA Manipulator End-Effector Velocity:
 [0.  1.5 0.5 0.  0.  1. ]


## Question 5

#### Solve problem 3-7 in the textbook and also verify your hand-derived answers using the code in Task 3.

In [21]:
d1, d2, d3 = 1, 2, 3  # Example values for d1, d2, and d3
dh_params = np.array([
    [0, d1, 0, -np.pi/2],
    [0, d2, 0, -np.pi/2],
    [0, d3, 0, 0]
])


Jacobian = calculate_jacobian(num_links, dh_params)
end_position = end_effector_position(num_links, dh_params)
end_velocity = end_effector_velocity(num_links, dh_params, joint_velocities)

print("Three-link Cartesian Manipulator Jacobian:\n", Jacobian)
print()
print("Three-link Cartesian Manipulator End-Effector Position:\n", end_position)
print()
print("Three-link Cartesian Manipulator End-Effector Velocity:\n", end_velocity)

Three-link Cartesian Manipulator Jacobian:
 [[ 1.0000000e+00  2.0000000e+00  2.0000000e+00]
 [ 0.0000000e+00 -0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 1.0000000e+00  1.2246468e-16  1.2246468e-16]
 [ 6.1232340e-17 -1.0000000e+00 -1.0000000e+00]]

Three-link Cartesian Manipulator End-Effector Position:
 [ 0.  2. -2.]

Three-link Cartesian Manipulator End-Effector Velocity:
 [ 2.5  0.   0.   0.   0.5 -1. ]


## Question 6

#### Solve problem 3-8 in the textbook and also verify your hand-derived answers using the code in Task 3.

In [22]:
d1, a2, a3, d6 = 1, 1, 1, 1
theta_star = np.pi / 4  

dh_params = np.array([
    [0, d1, np.pi/2, theta_star],
    [a2, 0, 0, theta_star],
    [a3, 0, 0, theta_star],
    [-np.pi/2, 0, 0, theta_star],
    [np.pi/2, 0, 0, theta_star],
    [0, d6, 0, theta_star]
])

def calculate_jacobian(num_links, dh_params, joint_types=None):
    if joint_types is None:
        joint_types = ['R'] * num_links 
    
    J = np.zeros((6, num_links))  
    T = np.eye(4)  

    for i in range(num_links):
        theta, d, a, alpha = dh_params[i]
        T_link = dh_transform_matrix(theta, d, a, alpha)
        T = np.dot(T, T_link)
        
        if joint_types[i] == 'R':
            J[:3, i] = np.cross(T[:3, 2], T[:3, 3] - np.eye(4)[:3, 3])
            J[3:, i] = T[:3, 2]
        else:
            J[:3, i] = T[:3, 2]
            J[3:, i] = np.zeros(3)

    return J

Jacobian = calculate_jacobian(num_links, dh_params)
print("Jacobian:\n", Jacobian)


Jacobian:
 [[-0.70710678 -0.77015115 -0.10272831]
 [ 1.11072073 -0.23396411 -0.62031454]
 [ 1.11072073  1.2097506   0.16136526]
 [ 0.          0.59500984  0.96954504]
 [-0.70710678 -0.77015115 -0.10272831]
 [ 0.70710678  0.22984885  0.22232704]]


In [23]:
# Function for End-Effector Position
def end_effector_position(num_links, dh_params):
    T = np.eye(4) 

    for i in range(num_links):
        theta, d, a, alpha = dh_params[i]
        T_link = dh_transform_matrix(theta, d, a, alpha)
        T = np.dot(T, T_link)

    return T[:3, 3]

end_position = end_effector_position(num_links, dh_params)
print("End-Effector Position:\n", end_position)

End-Effector Position:
 [1.57079633 0.         1.        ]


In [24]:
num_links = 6
joint_velocities = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

# Function for End-Effector Velocity
def end_effector_velocity(num_links, dh_params, joint_velocities, joint_types=None):
    J = calculate_jacobian(num_links, dh_params, joint_types)
    velocity = np.dot(J, joint_velocities)
    return velocity

joint_velocities = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
end_velocity = end_effector_velocity(num_links, dh_params, joint_velocities)
print("End-Effector Velocity:\n", end_velocity)

End-Effector Velocity:
 [-0.27752112 -1.54474253 -0.16079965  1.60276363 -0.0873241   0.04097128]
